In [ ]:
from utils import *
from mtnn import *
from figure9 import *
import torch
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
feature = np.load('mtnn_data/train/feature.npy')
output = np.load('mtnn_data/train/output.npy')

In [ ]:
feature.shape

In [ ]:
neuron_order = feature[:,0,0]
feature = feature[:,:,1:]

In [ ]:
neurons = np.unique(neuron_order)
n_neurons = neurons.shape[0]
print('number of neurons: {}'.format(n_neurons))

In [ ]:
HIDDEN_SIZE = 8
INPUT_SIZE_DYNAMIC = feature.shape[-1]-static_idx.shape[0]
INPUT_SIZE_STATIC = static_idx.shape[0]
print(INPUT_SIZE_DYNAMIC, INPUT_SIZE_STATIC)

In [ ]:
# prev: hidden_size = 16, n_layers=2, batch_size=256, random_sliding=1~2*max
model = initialize_mtnn(n_neurons=n_neurons, 
                        input_size_static=INPUT_SIZE_STATIC, 
                        input_size_dynamic=INPUT_SIZE_DYNAMIC,
                        output_size=1, hidden_dim_static=HIDDEN_SIZE, 
                        hidden_dim_dynamic=HIDDEN_SIZE, n_layers=1)
remove_cov = None
only_keep_cov = None
best_epoch, loss_list, val_loss_list = run_train(model, 
                                                 'mtnn_data/train/feature.npy', 
                                                 'mtnn_data/train/output.npy', 
                                                 'mtnn_data/validation/feature.npy', 
                                                 'mtnn_data/validation/output.npy',
                                                 batch_size=512, n_epochs=100, lr=0.03, 
                                                 remove_cov=remove_cov, 
                                                 only_keep_cov=only_keep_cov)

In [ ]:
# plt.figure()
# plt.plot(loss_list[1:], color='k')
# plt.plot(val_loss_list[1:])

In [ ]:
preds_shape = np.load('mtnn_data/test/shape.npy')
obs = np.load('mtnn_data/test/output.npy')
test_feature = np.load('mtnn_data/test/feature.npy')
neu_list = np.load('mtnn_data/clusters.npy', allow_pickle=True)
sess_list = np.load('mtnn_data/session_info.npy', allow_pickle=True)
trial_list = np.load('mtnn_data/test/trials.npy', allow_pickle=True)

In [ ]:
cov_pair_list = [(None,None), (('all',),None), (None, 'noise')]
scores = dict()
for cov_pair in cov_pair_list:
    remove_cov = cov_pair[0]
    only_keep_cov = cov_pair[1]
    
    model = initialize_mtnn(n_neurons=n_neurons, 
                            input_size_static=INPUT_SIZE_STATIC, 
                            input_size_dynamic=INPUT_SIZE_DYNAMIC,
                            output_size=1, hidden_dim_static=HIDDEN_SIZE, 
                            hidden_dim_dynamic=HIDDEN_SIZE, n_layers=1)

    print('loading... '+f'trained_models/state_dict_rem={remove_cov}_keep={only_keep_cov}.pt')
    model.load_state_dict(torch.load(f'trained_models/state_dict_rem={remove_cov}_keep={only_keep_cov}.pt'))
    
    preds, test_loss = run_eval(model,'mtnn_data/test/feature.npy',
                                'mtnn_data/test/output.npy', 
                                remove_cov=remove_cov, 
                                only_keep_cov=only_keep_cov)
    print(f'test loss: {test_loss}')
    
    pred_list = []
    obs_list = []
    feature_list = []
    idx = 0
    for sh in preds_shape:
        n = sh[0]*sh[1]
        pred_list.append(preds[idx:idx+n].reshape(sh[:-1]))
        obs_list.append(obs[idx:idx+n].reshape(sh[:-1]))
        feature_list.append(test_feature[idx:idx+n].reshape(sh))
        idx += n
    
    for sess in range(len(sess_list)):
        score = compute_score(obs_list[sess], 
                              pred_list[sess], 
                              metric='r2', 
                              use_psth=False)['r2']
        scores[(cov_pair,sess)] = score

In [ ]:
for sess in range(len(sess_list)):
    n = obs_list[sess].shape[0]
    plt.figure()
    plt.scatter(np.ones(n), scores[(None,None),sess])
    plt.scatter(np.ones(n)*2, scores[(('all',),None),sess])
    plt.scatter(np.ones(n)*3, scores[(None,'noise'),sess])
    plt.show()

In [ ]:
# generate_figure_9(feature_list, pred_list, obs_list, 
#                 neu_list, sess_list, trial_list, which_sess=[1], savefig=False)